In [26]:
import os
import json
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from collections import defaultdict
from urllib.request import Request, urlopen


save_dir = '/home/zxx5113/IBM/data/cyberkg_IBM/save/mitre-defend'
os.makedirs(save_dir, exist_ok=True)

miti2def = defaultdict(list)
'''
{
    'miti code' : {
        'name': str,
        'def_list': str,
    }
}
'''


mitre_defence = defaultdict(dict)
'''
{
    'def code' : {
        'name': str,
        'link': str,
        'desc': str
    }
}
'''

web_prefix = 'https://d3fend.mitre.org/technique/d3f:'

with open('save/mitre-defend/map.txt', 'r') as f:
    for line in f.readlines()[1:]:
        line = line.strip()
        miti = line.split()[0]
        defs = ['D3'+ele.strip() for ele in line.split('D3')[1:] if ele.startswith('-')]
        miti2def[miti] = [ele.split()[0] for ele in defs]
        
        for ele in defs:
            def_code = ele.split()[0]
            def_name = ' '.join(ele.split()[1:])
            web_postfix = ''.join(ele.split()[1:])
            mitre_defence[def_code] = {
                'name': def_name,
                'link': web_prefix + web_postfix,
            }

for code, data in mitre_defence.items():
    link = data['link']
    html = BeautifulSoup(requests.get(link).content, 'html.parser')
    for div in html.find_all('div',  attrs={'class':'text-justify'}):
        p = div.find('p') # the 1st one
        definition = p.text
        mitre_defence[code]['desc'] = definition
        
with open(os.path.join(save_dir, 'miti2def.json'), 'w') as f: 
    json.dump(miti2def, f)
    
with open(os.path.join(save_dir, 'defence.json'), 'w') as f: 
    json.dump(mitre_defence, f)
print('Done')
        

Done
